In [ ]:
#!/usr/bin/env python3
# ==============================================================================
# GPU SERVER - FIREBASE EDITION (V2.4 - DATASET CREDENTIAL LOADER)
# VERSION: 2.4 - PARALLEL GPU CORES + DATASET AUTO-LOADER
# ==============================================================================
import subprocess
import sys
import os
import time
import uuid
import threading
import json
import random
import math
import urllib.request
import logging
import shutil
import glob
from datetime import datetime, timedelta, timezone

# ============================================================================
# CRITICAL: CHANGE THIS FOR EACH GPU SERVER
# ============================================================================
SERVER_ID = "server_1"  # ← Change to: server_1, server_2, server_3, or server_4
# ============================================================================

# DATASET CONFIGURATION
CREDENTIALS_DATASET = "firebase-credentials"  # ← Same dataset as CPU bot

# Optimize Memory
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
logging.getLogger("transformers").setLevel(logging.ERROR)

# ------------------------------------------
# CONFIGURATION
# ------------------------------------------
R2_ACCOUNT_ID = "4fa19e788a951ba2d879c18782ef8bf0"
R2_ACCESS_KEY_ID = "d66adcff67ac5b4eca609a662b80e742"
R2_SECRET_ACCESS_KEY = "1a10aca3049c176d85cf3ec3c4e4ae8c6b715a4d9b1e67a79acc8b94d3b3c660"
R2_BUCKET_NAME = "video-generation-storage"
R2_ENDPOINT_URL = f"https://{R2_ACCOUNT_ID}.r2.cloudflarestorage.com"

# FIREBASE DATABASE URL (will auto-detect from credentials)
FIREBASE_DATABASE_URL = None  # Will be set after loading credentials

# ------------------------------------------
# DEPENDENCY CHECK
# ------------------------------------------
def install_if_missing(package, import_name=None):
    if import_name is None:
        import_name = package
    try:
        __import__(import_name)
    except ImportError:
        print(f"⬇️ Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

if shutil.which('ffmpeg') is None:
    print("Installing FFmpeg...")
    subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL)
    subprocess.run(["apt-get", "install", "-y", "ffmpeg"], stdout=subprocess.DEVNULL)

install_if_missing("boto3")
install_if_missing("firebase-admin", "firebase_admin")
install_if_missing("openai-whisper", "whisper")
install_if_missing("moviepy")
install_if_missing("unidecode")
install_if_missing("Pillow", "PIL")

try:
    import diffusers
    import transformers
    import accelerate
except ImportError:
    print("⬇️ Installing AI Libraries...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "diffusers", "transformers", "accelerate", "safetensors", "xformers"])

# ------------------------------------------
# IMPORTS
# ------------------------------------------
import torch
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler, AutoencoderKL
import boto3
from botocore.config import Config
import firebase_admin
from firebase_admin import credentials, db
import whisper
from PIL import Image, ImageFont, ImageDraw

# ==============================================================================
# FIREBASE CREDENTIAL AUTO-LOADER FROM KAGGLE DATASET
# ==============================================================================

def load_firebase_credentials_from_dataset():
    """
    Automatically loads ALL Firebase JSON credential files from Kaggle dataset.
    Returns list of config dictionaries with auto-failover support.
    """
    configs = []
    dataset_path = f"/kaggle/input/{CREDENTIALS_DATASET}"
    
    print("\n" + "="*70)
    print(f"🔍 [{SERVER_ID}] LOADING FIREBASE CREDENTIALS FROM DATASET")
    print("="*70)
    print(f"📁 Dataset: {CREDENTIALS_DATASET}")
    print(f"📂 Path: {dataset_path}\n")
    
    # Check if dataset exists
    if not os.path.exists(dataset_path):
        print(f"⚠️  Dataset not found: {dataset_path}")
        print("\n💡 SETUP INSTRUCTIONS:")
        print("   Make sure you added the firebase-credentials dataset to this notebook!")
        print("   (Should be the same dataset as CPU bot)")
        return []
    
    # Find all JSON files
    json_files = glob.glob(f"{dataset_path}/*.json")
    
    if not json_files:
        print(f"⚠️  No .json files found in {dataset_path}")
        return []
    
    print(f"✅ Found {len(json_files)} credential file(s):\n")
    
    # Load each file
    for json_file in sorted(json_files):
        try:
            with open(json_file, 'r') as f:
                cred_data = json.load(f)
            
            # Validate required fields
            if 'project_id' not in cred_data or 'private_key' not in cred_data:
                print(f"  ⚠️  Skipping {os.path.basename(json_file)}: Missing required fields")
                continue
            
            # Build database URL from project_id
            project_id = cred_data['project_id']
            database_url = f"https://{project_id}-default-rtdb.firebaseio.com"
            
            configs.append({
                'name': os.path.basename(json_file),
                'url': database_url,
                'cred': cred_data,
                'project_id': project_id
            })
            
            print(f"  ✅ {os.path.basename(json_file)}")
            print(f"     📌 Project: {project_id}")
            
        except json.JSONDecodeError:
            print(f"  ❌ Invalid JSON: {os.path.basename(json_file)}")
        except Exception as e:
            print(f"  ❌ Error: {os.path.basename(json_file)} - {e}")
    
    print("\n" + "="*70)
    print(f"📊 Total valid credentials: {len(configs)}")
    print("="*70 + "\n")
    
    return configs

def initialize_firebase_with_failover(configs):
    """
    Tries each credential until one works (auto-failover).
    Returns True if connected, False otherwise.
    """
    global FIREBASE_DATABASE_URL
    
    if not configs:
        print("⚠️  No credentials available to try!")
        return False
    
    print("="*70)
    print(f"🔧 [{SERVER_ID}] CONNECTING TO FIREBASE (AUTO-FAILOVER)")
    print("="*70 + "\n")
    
    for idx, config in enumerate(configs, 1):
        print(f"[Attempt {idx}/{len(configs)}] Trying: {config['name']}")
        print(f"  📌 Project: {config['project_id']}")
        
        try:
            # Check if already initialized
            try:
                app = firebase_admin.get_app()
                print(f"  ✅ Firebase already initialized (reusing connection)")
                FIREBASE_DATABASE_URL = config['url']
                return True
            except ValueError:
                pass  # Not initialized yet, proceed
            
            # Try to initialize with this credential
            cred = credentials.Certificate(config['cred'])
            firebase_admin.initialize_app(cred, {
                'databaseURL': config['url']
            })
            
            # Test connection
            test_ref = db.reference(f'/servers/{SERVER_ID}/heartbeat')
            test_ref.set({
                'timestamp': time.time(),
                'status': 'initializing',
                'server_id': SERVER_ID
            })
            
            FIREBASE_DATABASE_URL = config['url']
            
            print(f"  ✅ CONNECTION SUCCESSFUL!")
            print(f"  🎉 Using credential: {config['name']}")
            print(f"  🌐 Database: {config['url']}\n")
            return True
            
        except Exception as e:
            error_msg = str(e)
            if len(error_msg) > 150:
                error_msg = error_msg[:150] + "..."
            print(f"  ❌ Failed: {error_msg}")
            
            # Clean up failed attempt
            try:
                app = firebase_admin.get_app()
                firebase_admin.delete_app(app)
            except:
                pass
            
            if idx < len(configs):
                print(f"  🔄 Trying next credential...\n")
            else:
                print(f"  ⚠️  No more credentials to try\n")
    
    return False

# ==============================================================================
# FIREBASE INITIALIZATION
# ==============================================================================
print("\n" + "="*70)
print(f"🚀 [{SERVER_ID}] FIREBASE INITIALIZATION STARTING...")
print("="*70 + "\n")

# Load credentials from dataset
FIREBASE_CONFIGS = load_firebase_credentials_from_dataset()

if FIREBASE_CONFIGS:
    # Use dataset credentials with failover
    success = initialize_firebase_with_failover(FIREBASE_CONFIGS)
    
    if not success:
        print("\n" + "="*70)
        print(f"❌ [{SERVER_ID}] ALL DATASET CREDENTIALS FAILED!")
        print("="*70)
        print("\n⚠️  TROUBLESHOOTING:")
        print("  1. Check if credentials are revoked in Firebase Console")
        print("  2. Verify dataset is properly added to this notebook")
        print("  3. Check if dataset contains valid JSON files")
        print("\n")
        sys.exit(1)
else:
    print("\n" + "="*70)
    print(f"❌ [{SERVER_ID}] NO CREDENTIALS FOUND!")
    print("="*70)
    print("\n⚠️  Make sure you added the firebase-credentials dataset!")
    print("\n")
    sys.exit(1)

print("="*70)
print(f"✅ [{SERVER_ID}] FIREBASE READY!")
print("="*70 + "\n")

# ------------------------------------------
# FONT MANAGEMENT
# ------------------------------------------
FONT_DIR = "./fonts"
SUBTITLE_FONTS = {}

FONT_CONFIG = {
    'latin': {'file': 'NotoSans-Bold.ttf', 'url': 'https://github.com/notofonts/notofonts.github.io/raw/main/fonts/NotoSans/hinted/ttf/NotoSans-Bold.ttf', 'family_name': 'Noto Sans', 'weight': 'Bold'},
    'devanagari': {'file': 'NotoSansDevanagari-Bold.ttf', 'url': 'https://github.com/notofonts/notofonts.github.io/raw/main/fonts/NotoSansDevanagari/hinted/ttf/NotoSansDevanagari-Bold.ttf', 'family_name': 'Noto Sans Devanagari', 'weight': 'Bold'},
    'arabic': {'file': 'NotoSansArabic-Bold.ttf', 'url': 'https://github.com/notofonts/notofonts.github.io/raw/main/fonts/NotoSansArabic/hinted/ttf/NotoSansArabic-Bold.ttf', 'family_name': 'Noto Sans Arabic', 'weight': 'Bold'},
    'cjk': {'file': 'NotoSansCJKsc-Bold.otf', 'url': 'https://github.com/notofonts/noto-cjk/raw/main/Sans/OTF/SimplifiedChinese/NotoSansCJKsc-Bold.otf', 'family_name': 'Noto Sans CJK SC', 'weight': 'Bold'}
}

def setup_fonts():
    if not os.path.exists(FONT_DIR):
        os.makedirs(FONT_DIR)
    print("⬇️ Setting up fonts...")
    for script, config in FONT_CONFIG.items():
        path = os.path.join(FONT_DIR, config['file'])
        if not os.path.exists(path):
            try:
                urllib.request.urlretrieve(config['url'], path)
            except Exception as e:
                print(f"❌ Font download failed ({script}): {e}")
                continue
        if os.path.exists(path):
            SUBTITLE_FONTS[script] = path

setup_fonts()

# ------------------------------------------
# SUBTITLE ENGINE
# ------------------------------------------
def detect_script(text):
    if not text or not text.strip(): return 'latin'
    counts = {'devanagari': 0, 'arabic': 0, 'cjk': 0, 'latin': 0}
    for char in text:
        code = ord(char)
        if 0x0900 <= code <= 0x0DFF: counts['devanagari'] += 1
        elif 0x0600 <= code <= 0x077F: counts['arabic'] += 1
        elif 0x4E00 <= code <= 0x9FFF or 0x3040 <= code <= 0x30FF: counts['cjk'] += 1
        else: counts['latin'] += 1
    max_script = max(counts.items(), key=lambda x: x[1])
    return max_script[0] if max_script[1] > 0 else 'latin'

def get_font_for_text(text):
    script = detect_script(text)
    mapping = {'devanagari': 'devanagari', 'arabic': 'arabic', 'cjk': 'cjk'}
    key = mapping.get(script, 'latin')
    if key in SUBTITLE_FONTS:
        return SUBTITLE_FONTS[key], FONT_CONFIG[key], script
    return SUBTITLE_FONTS.get('latin'), FONT_CONFIG.get('latin'), 'latin'

def time_to_ass(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    cs = int((seconds % 1) * 100)
    return f"{h}:{m:02d}:{s:02d}.{cs:02d}"

def calculate_text_width(text, font_path, font_size):
    try:
        font = ImageFont.truetype(font_path, font_size)
        dummy = ImageDraw.Draw(Image.new("RGBA", (1, 1)))
        bbox = dummy.textbbox((0, 0), text, font=font)
        return bbox[2] - bbox[0]
    except:
        return len(text) * font_size * 0.6

def create_ass_event_for_page(page_lines, font_size, events, is_cjk, is_latin):
    if not page_lines: return
    start_time = page_lines[0][0]['start']
    end_time = page_lines[-1][-1]['end']
    
    def process_word(word_obj):
        raw = word_obj.get('word', '').strip()
        return raw.upper() if is_latin else raw
    
    separator = '' if is_cjk else ' '
    full_text_lines = [separator.join([process_word(w) for w in line]) for line in page_lines]
    full_text = '\\N'.join(full_text_lines)
    
    events.append(f"Dialogue: 0,{time_to_ass(start_time)},{time_to_ass(end_time)},BG,,0,0,0,,{{\\bord15\\shad0\\1c&H000000&\\1a&H4D&}}{full_text}")
    
    for line_idx, line in enumerate(page_lines):
        for word_idx, word_obj in enumerate(line):
            w_start = word_obj.get('start', start_time)
            w_end = word_obj.get('end', end_time)
            highlight_lines = []
            for l_idx, l in enumerate(page_lines):
                line_words = []
                for w_idx, w in enumerate(l):
                    disp = process_word(w)
                    if l_idx == line_idx and w_idx == word_idx:
                        line_words.append(f"{{\\bord8\\3c&HD30093&\\3a&H4D&\\shad2\\4c&H000000&\\4a&H00&}}{disp}{{\\bord0\\shad0}}")
                    else:
                        line_words.append(disp)
                highlight_lines.append(separator.join(line_words))
            hl_text = '\\N'.join(highlight_lines)
            events.append(f"Dialogue: 1,{time_to_ass(w_start)},{time_to_ass(w_end)},PurpleBG,,0,0,0,,{hl_text}")
    
    events.append(f"Dialogue: 2,{time_to_ass(start_time)},{time_to_ass(end_time)},Default,,0,0,0,,{full_text}")
    
    for line_idx, line in enumerate(page_lines):
        for word_idx, word_obj in enumerate(line):
            w_start = word_obj.get('start', start_time)
            w_end = word_obj.get('end', end_time)
            top_lines = []
            for l_idx, l in enumerate(page_lines):
                line_words = []
                for w_idx, w in enumerate(l):
                    disp = process_word(w)
                    if l_idx == line_idx and w_idx == word_idx:
                        line_words.append(f"{{\\c&HFFFFFF&}}{disp}")
                    else:
                        line_words.append(disp)
                top_lines.append(separator.join(line_words))
            joined_top_lines = '\\N'.join(top_lines)
            events.append(f"Dialogue: 3,{time_to_ass(w_start)},{time_to_ass(w_end)},Default,,0,0,0,,{joined_top_lines}")

def generate_clip_ass_subtitles(subtitle_words, width, height, output_file):
    if not subtitle_words: return False
    
    all_text_raw = "".join([w.get('word', '') for w in subtitle_words])
    font_path, font_config, script = get_font_for_text(all_text_raw)
    font_family = font_config['family_name']
    is_cjk = script == 'cjk'

    processed_words = []
    if is_cjk:
        for w in subtitle_words:
            text = w.get('word', '').strip()
            start = w.get('start', 0)
            end = w.get('end', 0)
            duration = end - start
            length = len(text)
            if length > 1:
                char_duration = duration / length
                for i, char in enumerate(text):
                    char_start = start + (i * char_duration)
                    char_end = start + ((i + 1) * char_duration)
                    processed_words.append({'word': char, 'start': char_start, 'end': char_end})
            elif length == 1:
                processed_words.append(w)
    else:
        processed_words = subtitle_words

    aspect = width / height
    if aspect < 0.8:
        ref_h, ref_f, ref_pos, pos_offset = 1280, 63, 0.75, 0
    elif aspect > 1.5:
        ref_h, ref_f, ref_pos, pos_offset = 720, 72, 0.85, 38
    else:
        ref_h, ref_f, ref_pos, pos_offset = 1080, 68, 0.80, 0
        
    font_size = int(ref_f * (height / ref_h))
    ref_position = int(height * ref_pos) + pos_offset
    margin_v = height - ref_position
    
    header = f"""[Script Info]
ScriptType: v4.00+
PlayResX: {width}
PlayResY: {height}
WrapStyle: 0
ScaledBorderAndShadow: yes
[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,{font_family},{font_size},&H00FFFFFF,&H000000FF,&H00000000,&HB3000000,-1,0,0,0,100,100,0,0,1,0,0,2,20,20,{margin_v},1
Style: BG,{font_family},{font_size},&H00FFFFFF,&H000000FF,&H00000000,&HB3000000,-1,0,0,0,100,100,0,0,1,0,0,2,20,20,{margin_v},1
Style: PurpleBG,{font_family},{font_size},&H00FFFFFF,&H000000FF,&H00000000,&HB4D30093,-1,0,0,0,100,100,0,0,1,3,2,2,20,20,{margin_v},1
[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    events = []
    is_latin = script == 'latin'
    max_line_width = width * 0.90
    space_width = 0 if is_cjk else int(font_size * 0.25)
    
    current_page_lines = []
    current_line = []
    current_line_width = 0
    
    for word_obj in processed_words:
        raw = word_obj.get('word', '').strip()
        disp = raw.upper() if is_latin else raw
        w_width = calculate_text_width(disp, font_path, font_size) + 10
        
        if current_line_width + w_width + space_width > max_line_width:
            if current_line: current_page_lines.append(current_line)
            if len(current_page_lines) >= 2:
                create_ass_event_for_page(current_page_lines, font_size, events, is_cjk, is_latin)
                current_page_lines = []
            current_line = [word_obj]
            current_line_width = w_width + space_width
        else:
            current_line.append(word_obj)
            current_line_width += w_width + space_width
            
    if current_line: current_page_lines.append(current_line)
    if current_page_lines: create_ass_event_for_page(current_page_lines, font_size, events, is_cjk, is_latin)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(header + "\n".join(events))
    return True

# ------------------------------------------
# R2 CLIENT
# ------------------------------------------
print("🔧 Setting up R2...")
s3_client = boto3.client(
    's3',
    endpoint_url=R2_ENDPOINT_URL,
    aws_access_key_id=R2_ACCESS_KEY_ID,
    aws_secret_access_key=R2_SECRET_ACCESS_KEY,
    config=Config(signature_version='s3v4'),
    region_name='auto'
)
print("✅ R2 ready")

# ------------------------------------------
# FIREBASE HEARTBEAT
# ------------------------------------------
def heartbeat_worker():
    print(f"💓 Heartbeat started for {SERVER_ID}...")
    while True:
        try:
            db.reference(f'/servers/{SERVER_ID}/heartbeat').set({
                'timestamp': time.time(),
                'status': 'active',
                'server_id': SERVER_ID
            })
            time.sleep(5)
        except Exception as e:
            print(f"💓 Heartbeat error: {e}")
            time.sleep(10)

threading.Thread(target=heartbeat_worker, daemon=True).start()

# ------------------------------------------
# WHISPER MODEL
# ------------------------------------------
print("⏳ Loading Whisper Model on GPU...")
WHISPER_MODEL = whisper.load_model("base", device="cuda")
print("✅ Whisper model loaded")

# ------------------------------------------
# TRANSCRIPTION FUNCTIONS
# ------------------------------------------
def format_time_ref(seconds):
    m = int(seconds // 60)
    s = int(seconds % 60)
    return f"{m:02d}:{s:02d}"

def create_prompts_txt(raw_result, duration):
    try:
        from unidecode import unidecode
        num_clips = math.ceil(duration / 6.0)
        bins = [""] * num_clips
        all_words = []
        
        INDIC_LANGS = {'hi', 'bn', 'gu', 'kn', 'ml', 'mr', 'pa', 'ta', 'te', 'ur', 'sd', 'ne'}
        detected_lang = raw_result.get('language', 'en')
        should_transliterate = detected_lang in INDIC_LANGS
        
        for segment in raw_result['segments']:
            if 'words' in segment:
                for w in segment['words']:
                    w_text = unidecode(w['word']) if should_transliterate else w['word']
                    all_words.append({'word': w_text, 'start': w['start']})
        
        if all_words:
            for w in all_words:
                start = w['start']
                idx = int(start // 6)
                if idx < num_clips:
                    bins[idx] += w['word'].strip() + " "
        else:
            for segment in raw_result['segments']:
                start = segment['start']
                idx = int(start // 6)
                txt = segment['text'].strip()
                if should_transliterate:
                    txt = unidecode(txt)
                if idx < num_clips:
                    bins[idx] += txt + " "
        
        lines = []
        for i in range(num_clips):
            start_str = format_time_ref(i * 6)
            end_str = format_time_ref((i + 1) * 6)
            text_content = bins[i].strip() if bins[i].strip() else "[No speech]"
            lines.append(f"[{i+1}] ({start_str} - {end_str})")
            lines.append(f"Text: {text_content}")
            lines.append("")
            lines.append("-" * 20)
            lines.append("")
        
        return "\n".join(lines)
    except Exception as e:
        print(f"❌ Prompts generation error: {e}")
        return None

def create_subtitles_ass(raw_result):
    try:
        from unidecode import unidecode
        INDIC_LANGS = {'hi', 'bn', 'gu', 'kn', 'ml', 'mr', 'pa', 'ta', 'te', 'ur', 'sd', 'ne'}
        detected_lang = raw_result.get('language', 'en')
        should_transliterate = detected_lang in INDIC_LANGS
        
        ass_lines = [
            "[Script Info]",
            "Title: Generated Subtitles",
            "ScriptType: v4.00+",
            "WrapStyle: 0",
            "ScaledBorderAndShadow: yes",
            "YCbCr Matrix: None",
            "",
            "[V4+ Styles]",
            "Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding",
            "Style: Default,Arial,20,&H00FFFFFF,&H000000FF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,2,2,10,10,10,1",
            "",
            "[Events]",
            "Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text",
            ""
        ]
        
        def format_ass_time(seconds):
            h = int(seconds // 3600)
            m = int((seconds % 3600) // 60)
            s = int(seconds % 60)
            cs = int((seconds % 1) * 100)
            return f"{h}:{m:02d}:{s:02d}.{cs:02d}"
        
        for segment in raw_result['segments']:
            start_time = format_ass_time(segment['start'])
            end_time = format_ass_time(segment['end'])
            text = segment['text'].strip()
            
            if should_transliterate:
                text = unidecode(text)
            
            text = text.replace('\\', '\\\\').replace('\n', '\\N')
            ass_lines.append(f"Dialogue: 0,{start_time},{end_time},Default,,0,0,0,,{text}")
        
        return "\n".join(ass_lines)
    except Exception as e:
        print(f"❌ Subtitles generation error: {e}")
        return None

def transcribe_audio_on_gpu(audio_path):
    try:
        print(f"🎙️ Transcribing with Whisper (GPU)...")
        result = WHISPER_MODEL.transcribe(audio_path, language=None, word_timestamps=True, verbose=False)
        print(f"✅ Transcription complete: {len(result['segments'])} segments")
        return result
    except Exception as e:
        print(f"❌ Transcription error: {e}")
        return None

def process_transcription_job(job_key, job_data):
    try:
        job = json.loads(job_data) if isinstance(job_data, str) else job_data
        job_id = job['job_id']
        audio_r2_key = job['audio_r2_key']
        
        print(f"\n🎙️ [TRANSCRIPTION] Processing: {job_id}")
        local_audio = f"/tmp/audio_{job_id}.mp3"
        print(f"   ⬇️ Downloading audio from R2...")
        s3_client.download_file(R2_BUCKET_NAME, audio_r2_key, local_audio)
        
        from moviepy.editor import AudioFileClip
        clip = AudioFileClip(local_audio)
        duration = clip.duration
        clip.close()
        
        print(f"   🎙️ Transcribing on GPU...")
        start_time = time.time()
        result = transcribe_audio_on_gpu(local_audio)
        
        if not result:
            db.reference(f'/transcriptions/failed/{job_id}').set({
                'error': 'Transcription failed',
                'timestamp': time.time()
            })
            db.reference(job_key).delete()
            os.remove(local_audio)
            return False
        
        transcribe_time = time.time() - start_time
        
        print(f"   📝 Generating prompts TXT...")
        prompts_txt = create_prompts_txt(result, duration)
        
        print(f"   📄 Generating subtitles ASS...")
        subtitles_ass = create_subtitles_ass(result)
        
        result_json = json.dumps(result)
        result_r2_key = f"transcriptions/{job_id}.json"
        prompts_r2_key = f"transcriptions/{job_id}_prompts.txt"
        subtitles_r2_key = f"transcriptions/{job_id}_subtitles.ass"
        
        print(f"   ⬆️ Uploading to R2...")
        s3_client.put_object(Bucket=R2_BUCKET_NAME, Key=result_r2_key, Body=result_json.encode('utf-8'), ContentType='application/json')
        s3_client.put_object(Bucket=R2_BUCKET_NAME, Key=prompts_r2_key, Body=prompts_txt.encode('utf-8'), ContentType='text/plain')
        s3_client.put_object(Bucket=R2_BUCKET_NAME, Key=subtitles_r2_key, Body=subtitles_ass.encode('utf-8'), ContentType='text/plain')
        
        completion_data = {
            "transcription_r2_key": result_r2_key,
            "prompts_r2_key": prompts_r2_key,
            "subtitles_r2_key": subtitles_r2_key,
            "duration": transcribe_time,
            "audio_duration": duration,
            "segments": len(result['segments']),
            "job_id": job_id,
            "server_id": SERVER_ID
        }
        
        db.reference(f'/transcriptions/completed/{job_id}').set(completion_data)
        db.reference(f'/notifications/transcriptions/{job_id}').set(completion_data)
        
        db.reference(job_key).delete()
        os.remove(local_audio)
        print(f"✅ [TRANSCRIPTION] Done! ⏱️ {transcribe_time:.1f}s")
        return True
        
    except Exception as e:
        print(f"❌ [TRANSCRIPTION] Failed: {e}")
        db.reference(f'/transcriptions/failed/{job_id}').set({
            'error': str(e),
            'timestamp': time.time()
        })
        try:
            db.reference(job_key).delete()
        except:
            pass
        return False

# ------------------------------------------
# VIDEO GENERATION
# ------------------------------------------
STEPS = 5
GUIDANCE_SCALE = 2.0

GPU_COUNT = torch.cuda.device_count()

def upload_to_r2(video_bytes, job_id, max_retries=3):
    for attempt in range(max_retries):
        try:
            video_key = f"videos/single/{job_id}.mp4"
            s3_client.put_object(Bucket=R2_BUCKET_NAME, Key=video_key, Body=video_bytes, ContentType='video/mp4')
            url = s3_client.generate_presigned_url('get_object', Params={'Bucket': R2_BUCKET_NAME, 'Key': video_key}, ExpiresIn=604800)
            return url
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
    return None

def image_to_video_clip(image, target_width, target_height, subtitle_path=None):
    unique_id = str(uuid.uuid4())
    img_path = f"temp_{unique_id}.png"
    vid_path = f"output_{unique_id}.mp4"

    try:
        image.save(img_path)
        w = (int(target_width) // 2) * 2
        h = (int(target_height) // 2) * 2
        
        zoom_curve = "1.1+0.1*cos(2*PI*on/180)"
        filter_chain = (
            f"scale={w}:{h}:flags=lanczos,"
            f"zoompan=z='{zoom_curve}':d=180:x='iw/2-(iw/zoom/2)':y='ih/2-(ih/zoom/2)':s={w}x{h},"
            f"vignette=PI/6"
        )
        
        if subtitle_path and os.path.exists(subtitle_path):
            abs_sub = os.path.abspath(subtitle_path).replace('\\', '/').replace(':', '\\:')
            abs_fonts = os.path.abspath(FONT_DIR).replace('\\', '/').replace(':', '\\:')
            filter_chain += f",subtitles='{abs_sub}':fontsdir='{abs_fonts}'"
        
        filter_chain += ",fps=30"
        
        cmd = [
            "ffmpeg", "-y", "-loop", "1", "-i", img_path, "-t", "6",
            "-vf", filter_chain,
            "-c:v", "libx264",
            "-preset", "veryfast",
            "-crf", "28",
            "-maxrate", "2M",
            "-bufsize", "4M",
            "-pix_fmt", "yuv420p",
            "-threads", "8",
            vid_path
        ]

        subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        
        if os.path.exists(vid_path):
            with open(vid_path, "rb") as f:
                return f.read()
        return None
    except Exception as e:
        print(f"Video generation error: {e}")
        return None
    finally:
        if os.path.exists(img_path): os.remove(img_path)
        if os.path.exists(vid_path): os.remove(vid_path)

def load_engine(device_id):
    print(f"⏳ Loading SDXL on GPU {device_id}...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "SG161222/RealVisXL_V4.0_Lightning",
        torch_dtype=torch.float16,
        variant="fp16"
    ).to(f"cuda:{device_id}")
    
    pipe.unet.to(memory_format=torch.channels_last)
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except: pass
    
    pipe.vae = AutoencoderKL.from_pretrained(
        "madebyollin/sdxl-vae-fp16-fix",
        torch_dtype=torch.float16
    ).to(f"cuda:{device_id}")

    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")
    pipe.set_progress_bar_config(disable=True)
    return pipe

pipelines = []
if GPU_COUNT > 0:
    for i in range(GPU_COUNT):
        pipelines.append(load_engine(i))

def get_safe_resolution(aspect_ratio_name):
    if aspect_ratio_name == "16:9": return (1152, 640, 1920, 1080)
    elif aspect_ratio_name == "9:16": return (640, 1152, 1080, 1920)
    elif aspect_ratio_name == "4:5": return (896, 1088, 1080, 1350)
    elif aspect_ratio_name == "16:7": return (1152, 512, 1920, 840)
    else: return (1024, 1024, 1024, 1024)

def process_single_job(job_data, pipeline, gpu_id):
    """Process a single video generation job"""
    job_id = job_data['job_id']
    prompt = job_data['prompt']
    ratio = job_data['ratio']
    subtitle_data = job_data.get('subtitle_data', [])
    
    print(f"🎬 [{SERVER_ID}/GPU{gpu_id}] Starting: {job_id}")
    
    start_time = time.time()
    ai_w, ai_h, final_w, final_h = get_safe_resolution(ratio)
    
    # Generate image
    neg = "text, watermark, signature, logo, writing, letters, font, typography, bad anatomy, bad hands, missing fingers, extra fingers, three hands, three legs, bad arms, missing legs, missing arms, poorly drawn face, bad face, fused face, cloned face, three crus, fused feet, fused thigh, extra crus, ugly, gross, sloppy, messy, blurry, low quality, duplicate, distortion, mutation, double head"
    
    image = pipeline(
        prompt,
        num_inference_steps=STEPS,
        guidance_scale=GUIDANCE_SCALE,
        width=ai_w,
        height=ai_h,
        negative_prompt=neg
    ).images[0]
    
    # Generate subtitle file if needed
    subtitle_path = None
    if subtitle_data:
        try:
            unique_sub_id = str(uuid.uuid4())
            temp_ass_path = f"temp_{unique_sub_id}.ass"
            success = generate_clip_ass_subtitles(subtitle_data, final_w, final_h, temp_ass_path)
            if success:
                subtitle_path = temp_ass_path
        except Exception as e:
            print(f"Subtitle gen failed: {e}")
    
    # Render video
    video_bytes = image_to_video_clip(image, target_width=final_w, target_height=final_h, subtitle_path=subtitle_path)
    
    if subtitle_path and os.path.exists(subtitle_path):
        os.remove(subtitle_path)
    
    if video_bytes:
        size_mb = len(video_bytes) / (1024 * 1024)
        r2_url = upload_to_r2(video_bytes, job_id)
        gen_time = time.time() - start_time
        
        if r2_url:
            result_data = {
                "r2_url": r2_url,
                "time": gen_time,
                "size_mb": size_mb,
                "job_id": job_id,
                "server_id": SERVER_ID,
                "gpu_id": gpu_id
            }
            
            db.reference(f'/servers/{SERVER_ID}/completed/{job_id}').set(result_data)
            db.reference(f'/notifications/videos/{job_id}').set(result_data)
            
            print(f"✅ [{SERVER_ID}/GPU{gpu_id}] Done! ⏱️ {gen_time:.1f}s | 💾 {size_mb:.2f}MB")
            return True
        else:
            db.reference(f'/servers/{SERVER_ID}/failed/{job_id}').set({
                'error': 'Upload failed',
                'timestamp': time.time()
            })
    else:
        db.reference(f'/servers/{SERVER_ID}/failed/{job_id}').set({
            'error': 'Video generation failed',
            'timestamp': time.time()
        })
    
    return False

# ============================================================================
# PARALLEL GPU WORKER - EACH GPU RUNS INDEPENDENTLY
# ============================================================================
def gpu_worker_loop(gpu_id, pipeline):
    """
    Each GPU runs this loop independently in parallel
    Both GPUs continuously fetch and process jobs
    """
    print(f"🚀 [{SERVER_ID}] GPU{gpu_id} Worker Started")
    
    while True:
        try:
            # Fetch job from Firebase queue
            queue_ref = db.reference(f'/servers/{SERVER_ID}/job_queue')
            jobs = queue_ref.get()
            
            if jobs and isinstance(jobs, dict):
                # Get first available job
                job_key = next(iter(jobs.keys()))
                job_data = jobs[job_key]
                
                # Atomic: Delete job from queue immediately (prevent other GPUs from taking it)
                queue_ref.child(job_key).delete()
                
                # Process the job on this GPU
                process_single_job(job_data, pipeline, gpu_id)
            else:
                # No jobs available, wait a bit
                time.sleep(0.5)
            
        except Exception as e:
            print(f"❌ [{SERVER_ID}] GPU{gpu_id} Error: {e}")
            time.sleep(2)

# ------------------------------------------
# MAIN LOOP - START PARALLEL GPU WORKERS
# ------------------------------------------
def main():
    print("\n" + "="*70)
    print(f"🚀 GPU SERVER: {SERVER_ID} (DATASET AUTO-LOADER)")
    print("="*70)
    print(f"✅ GPUs: {GPU_COUNT}")
    print(f"✅ Steps: {STEPS}")
    print(f"✅ Firebase: {FIREBASE_DATABASE_URL}")
    print(f"✅ Path: /servers/{SERVER_ID}")
    print(f"✅ Credentials: Loaded from dataset (auto-failover enabled)")
    print(f"✅ Mode: PARALLEL (Each GPU processes independently)")
    print("="*70 + "\n")
    
    # Start a worker thread for each GPU
    gpu_threads = []
    for gpu_id in range(GPU_COUNT):
        t = threading.Thread(
            target=gpu_worker_loop,
            args=(gpu_id, pipelines[gpu_id]),
            daemon=True
        )
        t.start()
        gpu_threads.append(t)
    
    print(f"🔥 {GPU_COUNT} GPU Workers Running in Parallel!\n")
    
    # Main thread handles transcription (priority) and shutdown
    while True:
        # Check shutdown time (3:30 PM and 11:00 PM IST)
        now_utc = datetime.now(timezone.utc)
        now_ist = now_utc + timedelta(hours=5, minutes=30)
        
        is_time1 = (now_ist.hour == 15 and now_ist.minute == 30)
        is_time2 = (now_ist.hour == 23 and now_ist.minute == 0)
        
        if is_time1 or is_time2:
            print(f"\n🛑 [{SERVER_ID}] Scheduled Shutdown: {now_ist.strftime('%I:%M %p')} IST")
            sys.exit(0)

        try:
            # Handle transcription jobs (priority over video generation)
            transcribe_ref = db.reference(f'/transcriptions/pending')
            transcribe_jobs = transcribe_ref.get()
            
            if transcribe_jobs and isinstance(transcribe_jobs, dict):
                job_key = next(iter(transcribe_jobs.keys()))
                job_data = transcribe_jobs[job_key]
                print(f"\n🔥 [{SERVER_ID}] PRIORITY: Transcription job found")
                process_transcription_job(f'/transcriptions/pending/{job_key}', job_data)
            
            time.sleep(1)
            
        except KeyboardInterrupt:
            print(f"\n🛑 [{SERVER_ID}] Shutting down...")
            break
        except Exception as e:
            print(f"❌ [{SERVER_ID}] Main loop error: {e}")
            time.sleep(5)

if __name__ == "__main__":
    main()

⬇️ Installing openai-whisper...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.8 MB/s eta 0:00:00
⬇️ Installing unidecode...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.2 MB/s eta 0:00:00


2026-01-05 02:48:59.985131: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767581340.197964      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767581340.263256      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767581340.771208      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767581340.771257      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767581340.771260      55 computation_placer.cc:177] computation placer alr


🚀 [server_1] FIREBASE INITIALIZATION STARTING...


🔍 [server_1] LOADING FIREBASE CREDENTIALS FROM DATASET
📁 Dataset: firebase-credentials
📂 Path: /kaggle/input/firebase-credentials

✅ Found 4 credential file(s):

  ✅ kaggle-a46bc-566779df174a.json
     📌 Project: kaggle-a46bc
  ✅ kaggle-a46bc-7716cbe39f8e.json
     📌 Project: kaggle-a46bc
  ✅ kaggle-a46bc-c137981f3dff.json
     📌 Project: kaggle-a46bc
  ✅ kaggle-a46bc-edb16e7c97f4.json
     📌 Project: kaggle-a46bc

📊 Total valid credentials: 4

🔧 [server_1] CONNECTING TO FIREBASE (AUTO-FAILOVER)

[Attempt 1/4] Trying: kaggle-a46bc-566779df174a.json
  📌 Project: kaggle-a46bc
  ✅ CONNECTION SUCCESSFUL!
  🎉 Using credential: kaggle-a46bc-566779df174a.json
  🌐 Database: https://kaggle-a46bc-default-rtdb.firebaseio.com

✅ [server_1] FIREBASE READY!

⬇️ Setting up fonts...
🔧 Setting up R2...
✅ R2 ready
💓 Heartbeat started for server_1...
⏳ Loading Whisper Model on GPU...


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 74.5MiB/s]


✅ Whisper model loaded
⏳ Loading SDXL on GPU 0...


...

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

⏳ Loading SDXL on GPU 1...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]


🚀 GPU SERVER: server_1 (DATASET AUTO-LOADER)
✅ GPUs: 2
✅ Steps: 5
✅ Firebase: https://kaggle-a46bc-default-rtdb.firebaseio.com
✅ Path: /servers/server_1
✅ Credentials: Loaded from dataset (auto-failover enabled)
✅ Mode: PARALLEL (Each GPU processes independently)

🚀 [server_1] GPU0 Worker Started
🚀 [server_1] GPU1 Worker Started
🔥 2 GPU Workers Running in Parallel!

🎬 [server_1/GPU1] Starting: job_1767581446_19331_2
🎬 [server_1/GPU0] Starting: job_1767581446_59632_5
✅ [server_1/GPU1] Done! ⏱️ 14.9s | 💾 0.48MB
✅ [server_1/GPU0] Done! ⏱️ 15.2s | 💾 0.36MB
🎬 [server_1/GPU1] Starting: job_1767581463_50946_8
🎬 [server_1/GPU0] Starting: job_1767581464_67909_11
✅ [server_1/GPU1] Done! ⏱️ 12.0s | 💾 0.49MB
🎬 [server_1/GPU1] Starting: job_1767581474_56872_14
✅ [server_1/GPU0] Done! ⏱️ 12.7s | 💾 0.35MB
🎬 [server_1/GPU0] Starting: job_1767581479_34404_17
✅ [server_1/GPU1] Done! ⏱️ 10.8s | 💾 0.50MB
🎬 [server_1/GPU1] Starting: job_1767581488_15081_20
✅ [server_1/GPU0] Done! ⏱️ 12.0s | 💾 0.31MB
🎬 [se